In [1]:
!pip install gensim pandas spacy openpyxl scikit-learn pyldavis

In [10]:
!python -m spacy download en_core_web_sm
!pip install sentence-transformers
!pip install umap-learn hdbscan
!pip install pandas wordcloud matplotlib pillow
!pip install bertopic
!pip install bertopic==0.10.0  # 모델 훈련에 사용한 버전
!pip install river

from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary # Import Dictionary from gensim.corpora
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from PIL import Image
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.vectorizers import OnlineCountVectorizer

from river import cluster
from river import stream

import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import spacy
import numpy as np
import plotly.graph_objects as go

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached bertopic-0.10.0-py2.py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel d

In [12]:
documents = df['키워드'].tolist()  # 분석할 텍스트 열 지정

# 2. 전처리 파이프라인
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# 3. 제외할 단어 리스트
# remove_words = {'인공지능', 'artificial', 'intelligence', 'artificial intelligence', 'Artificial', 'Intelligence', 'AI', '및', 'ai'}

# 4. 전처리 함수
def preprocess_text(text):
    if not isinstance(text, str):
        return []
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [token for token in tokens if token not in remove_words]  # <-- 여기 추가!
    return tokens

# 5. 전처리 실행
documents = [preprocess_text(doc) for doc in documents]

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)  # 숫자 등 다른 타입을 문자열로 변환
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

processed_docs = [preprocess_text(doc) for doc in documents]

# 6. DTM 생성
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# 7. LDA 모델 학습
lda_model = models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    passes=15,
    num_topics=10,  # 토픽 개수 10개로 지정
    random_state=42
)

# 8. 토픽 결과 시각화
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.save_html(vis, "/content/drive/MyDrive/data/보건의료_20250507-20250807.xlsx")

# 9. 토픽별 상위 단어 출력
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(f"토픽 {topic[0]}: {topic[1]}")

# 10. 토픽 결과를 데이터프레임으로 변환
df_topics = pd.DataFrame(topics, columns=['토픽번호', '상위단어'])
output_path = "/content/drive/MyDrive/data/보건의료_20250507-20250807.xlsx"
df_topics.to_excel(output_path, index=False)

# 11. 문서별 토픽 배정
doc_topics = [lda_model.get_document_topics(근조) for bow in corpus]

NameError: name 'df' is not defined

In [ ]:
##BERTopic

# 1. 원문 텍스트를 사용 (BERTopic은 토큰화된 결과 대신 원문 텍스트를 선호합니다)
raw_documents = [' '.join(doc) for doc in processed_docs] # 이미 토큰화된 문서를 다시 텍스트로 변환

# 2. BERTopic 모델 설정 (한국어 문서에 적합한 모델 사용)
vectorizer = CountVectorizer(stop_words="english") # 한국어의 경우 커스텀 불용어 리스트 사용 가능

topic_model = BERTopic(
    language="korean",
    nr_topics=10,
    vectorizer_model=vectorizer,
    min_topic_size=5,
    calculate_probabilities=True
)

# 3. 모델 학습 및 토픽 추출
topics, probs = topic_model.fit_transform(raw_documents)


# 기본 히트맵 생성
heatmap_fig = topic_model.visualize_heatmap(
    top_n_topics=10,
    width=800,
    height=800
)

# 사용자 정의 라벨 생성
custom_labels = [f"토픽 {i}" for i in range(10)]

# 축 레이블 수동 수정
heatmap_fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(10)),
        ticktext=custom_labels
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=list(range(10)),
        ticktext=custom_labels
    )
)

# 결과 출력 및 저장
heatmap_fig.show()
heatmap_fig.write_html("/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/논문보고서종합분석/분석결과/total/total_topic_heatmap.html")

# 1. 토픽 시각화
fig = topic_model.visualize_topics()
fig.write_html("/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/논문보고서종합분석/분석결과/total/total_bertopic_topics.html")

# 2. 토픽 분포 시각화
fig = topic_model.visualize_distribution(probs[0], min_probability=0.001)
fig.write_html("/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/논문보고서종합분석/분석결과/total/total_bertopic_distribution.html")

# 3. 학습 확인 (디버깅용)
print("Model fitted:", hasattr(topic_model, "topics_") and topic_model.topics_ is not None)

# 4. 학습이 확인된 후 시각화
if hasattr(topic_model, "topics_") and topic_model.topics_ is not None:
    fig = topic_model.visualize_hierarchy()
    fig.write_html("/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/논문보고서종합분석/분석결과/total/total_bertopic_hierarchy.html")
else:
    print("모델이 아직 학습되지 않았습니다.")

# 5. 토픽별 주요 단어 출력
#for topic_id in range(10):
#    print(f"토픽 {topic_id}: {topic_model.get_topic(topic_id)}")

In [ ]:
## wordcloud

!apt-get update -qq
!apt-get install fonts-nanum* -qq

%config InlineBackend.figure_format='retina'
sys_font=fm.findSystemFonts()

[f for f in sys_font if 'Nanum' in f]

import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import re

# 엑셀 파일 불러오기 (경로 수정 필요)
file_path = "/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/기사분석/dtotal.xlsx"  # 실제 파일 경로로 수정
df = pd.read_excel(file_path)

# '키워드' 컬럼의 텍스트 합치기
text = ' '.join(df['키워드'].astype(str).tolist())
text = text.replace(' 인공 지능 ', '인공지능')

# 제외할 단어들
excluded_words = ['인공지능', '인공', '지능', 'nan', '및', 'D', 'and', 'of', '연구', 'artificial', 'intelligence', 'Artificial', 'Intelligence', 'artificial intelligence', 'AI', '및', 'ai', '0', '1']

# 단어 분리 및 필터링
words = re.findall(r'\w+', text)
filtered_words = [word for word in words if word not in excluded_words]
filtered_text = ' '.join(filtered_words)
filtered_text = filtered_text.replace('\n', ' ').replace('\r', ' ')
filtered_text = ' '.join(filtered_text.split())  # 연속 공백 제거

import re

filtered_text = re.sub(r'[^가-힣\s]', '', filtered_text)  # 한글과 공백만 남기기
filtered_text = ' '.join(filtered_text.split())


# 워드클라우드 생성
wordcloud = WordCloud(
    font_path='/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf',  # 한글 폰트 경로
    width=800,
    height=600,
    background_color='white',
    max_words=200
).generate(filtered_text)

# 워드클라우드 시각화
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

plt.savefig('/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/기사종합분석/total_wordcloud.png', dpi=300, bbox_inches='tight', transparent=True)

plt.show()

# 단어 빈도수 계산
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

# 단어 빈도수 계산
word_counts = Counter(filtered_words)

# 상위 20개 단어 빈도수 출력
print("가장 많이 등장한 단어 (상위 20개):")
for word, count in word_counts.most_common(20):
    print(f"{word}: {count}회")

# 빈도수를 데이터프레임으로 변환하여 표 형태로 출력
df_word_counts = pd.DataFrame(word_counts.most_common(20), columns=['단어', '빈도수'])

# 표 형태로 출력
print("\n단어 빈도수 표:")
print(df_word_counts)

# 표 형식 스타일링 적용 (Jupyter Notebook 또는 Colab에서 사용)
display_table = df_word_counts.style.background_gradient(cmap='Blues')
display(display_table)

# 엑셀 파일로 바로 저장
df_word_counts.to_excel('/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/기사종합분석/total_word_frequency_table.xlsx', index=False)

# HTML 형식으로 표 저장 (옵션)
html_table = df_word_counts.to_html(index=False)
with open('/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/기사종합분석/total_word_frequency_table.html', 'w', encoding='utf-8') as f:
    f.write(html_table)

# 빈도수 그래프로 시각화 (막대 위에 빈도수 표시)
plt.figure(figsize=(12, 8))
words, counts = zip(*word_counts.most_common(20))
bars = plt.bar(words, counts, color='skyblue')

# 각 막대 위에 빈도수 표시
for bar, count in zip(bars, counts):
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # 막대의 중앙 x 좌표
        bar.get_height() + 0.2,            # 막대의 높이 바로 위
        str(count),                        # 표시할 텍스트 (빈도수)
        ha='center',                       # 텍스트 정렬 (가로)
        va='bottom',                       # 텍스트 정렬 (세로)
        fontsize=10                        # 글씨 크기
    )

# 그래프 제목 및 축 레이블 설정
plt.xticks(rotation=45, ha='right')
plt.title("단어 빈도수 (상위 20개)")
plt.xlabel("단어")
plt.ylabel("빈도수")
plt.tight_layout()

plt.savefig('/content/drive/MyDrive/Colab Notebooks/data/인공지능정책/기사종합분석/total_word_frequency_chart.png', dpi=300)

plt.show()
